# WEB SCRAPING 
De la página web de venta de zapatillas deportivas https://deportesmoya.es/1585-zapatillas-baloncesto vamos a extraer el nombre del producto, el precio rebajado, el precio original, el descuento y el enlace a cada producto.

In [ ]:
# pip install beautifulsoup4


In [14]:
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup # esta libreria es para extraer la información de la pagina web


In [47]:
# vamos a crearnos un diccionario donde luego nos guardaremos todos los resultados de nuestras búsquedas: 

resultados_zapatillas = {"producto":[], 
                        "precio_rebajado": [], 
                        "precio_original": [], 
                         "descuento": [], 
                        "enlaces": []}

In [2]:
url = "https://deportesmoya.es/1585-zapatillas-baloncesto" #url de la que sacaremos la información

In [3]:
res1= requests.get(url) #llamamos a la url

In [4]:
res1.status_code #comprobamos el status de la llamada

200

In [ ]:
# res1.content ver el contenido de la pagina

In [6]:
sopa = BeautifulSoup(res1.content, "html.parser")

In [8]:
type(sopa)

bs4.BeautifulSoup

In [9]:
productos = sopa.find_all("h3", {"class":"h3 product-title"}) # esta configuración es de acuerdo a la inspección de la pagina web

In [ ]:
productos

In [11]:
len(productos) #vemos la cantidad de productos extraidos

48

In [48]:
productos[0]

<h3 class="h3 product-title"><a href="https://deportesmoya.es/zapatillas-adidas-ownthegame-20-gris-dorado-hombre-126442.html#/3402-tallaje_calzado_unisex-40_2_3">Zapatillas adidas Ownthegame 2.0 gris dorado hombre</a></h3>

In [12]:
productos[0].getText() # nombre del producto

'Zapatillas adidas Ownthegame 2.0 gris dorado hombre'

In [50]:
resultados_zapatillas = {"producto": [], 
                        "precio_rebajado": [], 
                        "precio_original": [], 
                        "descuento": [], 
                        "enlaces": []}
for a in productos: 
    resultados_zapatillas["producto"].append(a.getText())


In [ ]:
resultados_zapatillas

In [22]:
#lista de precios
precios = sopa.find_all("span", {"class": "price"})

In [23]:
len(precios)

48

In [52]:
precios[9].getText() #vemos que al obtener el precio no esta limpio

'40,00\xa0€'

In [26]:
# limpiamos los resultados: 

patron = '\d+,\d+'

for precio in precios:
    resultados_zapatillas["precio_rebajado"].append(re.findall(patron, precio.getText())[0])

In [ ]:
resultados_zapatillas

In [53]:
precios_originales = sopa.select("span.regular-price") # Sacamos los precios originales

In [54]:
precios_originales[:2]

[<span class="regular-price"><span class="soy_price_principal">70</span><span class="soy_price_separador">,</span><span class="soy_price_secondary">00 €</span></span>,
 <span class="regular-price"><span class="soy_price_principal">100</span><span class="soy_price_separador">,</span><span class="soy_price_secondary">00 €</span></span>]

In [55]:
precios_originales[2].getText()

'45,00\xa0€'

In [56]:
resultados_zapatillas.keys()

dict_keys(['producto', 'precio_rebajado', 'precio_original', 'descuento', 'enlaces'])

In [57]:
for precio in precios_originales:
    resultados_zapatillas["precio_original"].append(re.findall(patron, precio.getText())[0])

In [ ]:
resultados_zapatillas

In [59]:
enlaces = sopa.find_all("div", {"class": "thumbnail-container"}) #Sacamos los enlaces

In [ ]:
enlaces[0]

In [61]:
enlaces[0].getText() #vemos que de esta forma no lo podemos obtener revisamos como configurarlo

'Zapatillas adidas Ownthegame 2.0 gris dorado hombrePrecio59,50\xa0€Precio normal70,00\xa0€-15% \n\n\n\n\n\n\ncancel El producto ha sido añadido a su lista de deseos correctamente.\n\n-15%\nCantidad\n\n\n\n\n\n\nAñadir a la cesta\nAñadir a la cesta\n\n\n\n\n\n\n\n\n\ncancel El producto ha sido añadido a su lista de deseos correctamente.\n\nGuía de tallas\n\n\n\n\n\n\n'

In [62]:
enlaces[0].find("a").get("href") #configuramos

'https://deportesmoya.es/zapatillas-adidas-ownthegame-20-gris-dorado-hombre-126442.html#/3402-tallaje_calzado_unisex-40_2_3'

In [63]:
# ahora sacams los enlaces
for enlace in enlaces: 
    resultados_zapatillas["enlaces"].append(enlace.find("a").get("href"))

In [64]:
descuento = sopa.find_all("li", {"class": 'product-flag discount'}) # sacamos el descuento

In [65]:
descuento[:2] 

[<li class="product-flag discount">-15%</li>,
 <li class="product-flag discount">-15%</li>]

In [66]:
descuento[2].getText()

'-15%'

In [67]:
for desc in descuento:
    resultados_zapatillas["descuento"].append(desc.getText())

Convertimos en dataframe

In [ ]:
df_zapatillas = pd.DataFrame(resultados_zapatillas)